# Telegram-боты: примеры

## Хранение данных в коде

Код ниже предлагает выбрать тему для получения ссылки на конспект этой темы.

```python
import telebot

bot = telebot.TeleBot(TOKEN)

link_1 = {'name': 'Ввод-вывод, математические операции', 'link': 'https://github.com/naleont/8-III/blob/main/01_input-output_math-operations.ipynb'}
link_2 = {'name': 'Строки', 'link': 'https://github.com/naleont/8-III/blob/main/02_strings.ipynb'}
link_3 = {'name': 'Условный оператор', 'link': 'https://github.com/naleont/8-III/blob/main/03_conditions.ipynb'}


@bot.message_handler(commands=['start'])
def start_command(message):
    markup = telebot.types.InlineKeyboardMarkup(row_width=1) # Добавлено максимальное число кнопок в ряду
    item_1 = telebot.types.InlineKeyboardButton(text='Ввод-вывод, математические операции', callback_data='theory_1')
    item_2 = telebot.types.InlineKeyboardButton(text='Строки', callback_data='theory_2')
    item_3 = telebot.types.InlineKeyboardButton(text='Условный оператор', callback_data='theory_3')
    markup.add(item_1, item_2, item_3)
    bot.send_message(message.chat.id, 'Выбери тему', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'theory_1')
def theory_1(call):
    name = link_1['name']
    link = link_1['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

@bot.callback_query_handler(func=lambda call: call.data == 'theory_2')
def theory_2(call):
    name = link_2['name']
    link = link_2['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

@bot.callback_query_handler(func=lambda call: call.data == 'theory_3')
def theory_3(call):
    name = link_3['name']
    link = link_3['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

bot.polling(none_stop=True)
```



А это - тот же бот, написанный более эффективным кодом:

```python
import telebot

bot = telebot.TeleBot(TOKEN)

# Объединили данные по всей теории в список вместо использования отдельной переменной для каждой темы
links = [{'id': '1', 'name': 'Ввод-вывод, математические операции', 'link': 'https://github.com/naleont/8-III/blob/main/01_input-output_math-operations.ipynb'},
{'id': '2', 'name': 'Строки', 'link': 'https://github.com/naleont/8-III/blob/main/02_strings.ipynb'},
{'id': '3', 'name': 'Условный оператор', 'link': 'https://github.com/naleont/8-III/blob/main/03_conditions.ipynb'},
{'id': '4', 'name': 'Больше об условиях и булевых данных', 'link': 'https://github.com/naleont/8-III/blob/main/04_more-conditions_boolean-type.ipynb'},
{'id': '5', 'name': 'Списки, цикл while', 'link': 'https://github.com/naleont/8-III/blob/main/05_lists_while-loop.ipynb'},
{'id': '6', 'name': 'Цикл for, словари', 'link': 'https://github.com/naleont/8-III/blob/main/06_for-loop_dictionaries.ipynb'},
{'id': '7', 'name': 'Функции, файлы', 'link': 'https://github.com/naleont/8-III/blob/main/07_files_functions.ipynb'}]


@bot.message_handler(commands=['start'])
def start_command(message):
    markup = telebot.types.InlineKeyboardMarkup(row_width=1)
    # Для получения данных по всем темам проходимся циклом по списку
    for link in links:
        # ID темы добавляем к имени колбэка
        item = telebot.types.InlineKeyboardButton(text=link['name'], callback_data='theory_' + link['id'])
        markup.add(item)
    bot.send_message(message.chat.id, 'Выбери тему', reply_markup=markup)

# В слушателе ловим только часть колбэка, одинаковую для всех тем
@bot.callback_query_handler(func=lambda call: call.data.split('_')[0] == 'theory')
def button_1(call):
    # Конкретную тему определяем по второй части колбэка
    link_id = call.data.split('_')[1]
    # Фильтруем список тем по ID нужной темы
    link_data = list(filter(lambda x: x['id'] == link_id, links))[0]
    name = link_data['name']
    link = link_data['link']
    bot.send_message(call.message.chat.id, f'[{name}]({link})', parse_mode='Markdown')

bot.polling(none_stop=True)
```

А еще удобнее будет, если мы переменную links со списком словарей будем хранить в отдельном файле и импортировать в код бота, как мы это делали с переменной, хранящей токен. Например, мы можем переменную `links` сохранить в файле `bot_links.py`. Если этот файл хранить в одной папке с кодом бота, импорт переменной в код будет выглядеть так:

```python
from bot_links import links
```

## Пример бота с опросом

А это - пример бота с опросом. [Здесь](https://github.com/naleont/dg_8_II/blob/main/10_tg-bots.pdf) - блок-схема с логикой работы. Сначала изучите ее, а потом - код ниже.

```python

import telebot

bot = telebot.TeleBot(TOKEN)

data = {}

def school_student(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Да', callback_data='school_student-yes')
    item_2 = telebot.types.InlineKeyboardButton(text='Нет', callback_data='school_student-no')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Вы учитесь в школе?', reply_markup=markup)


def school_class(message):
    markup = telebot.types.ForceReply(input_field_placeholder='10', selective=False)
    bot.register_next_step_handler(message, school_class_handler)
    bot.send_message(message.chat.id, 'В каком вы классе?', reply_markup=markup)


def school_class_handler(message):
    data['school_class'] = message.text
    good_sleep(message)


def good_sleep(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Да', callback_data='good_sleep-yes')
    item_2 = telebot.types.InlineKeyboardButton(text='Нет', callback_data='good_sleep-no')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Вы сегодня выспались?', reply_markup=markup)


@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id,
    'Спасибо, что согласились принять участие в опросе! Он займет всего пару минут. Все данные хранятся анонимно.')
    school_student(message)


@bot.callback_query_handler(func=lambda call: call.data.split('-')[0] == 'school_student')
def school_student_handler(call):
    reply = call.data.split('-')[1]
    data['school_student'] = reply
    if reply == 'yes':
        school_class(call.message)
    else:
        data['school_class'] = ''
        good_sleep(call.message)


@bot.callback_query_handler(func=lambda call: call.data.split('-')[0] == 'good_sleep')
def good_sleep_handler(call):
    reply = call.data.split('-')[1]
    data['good_sleep'] = reply
    if reply == 'yes':
        bot.send_message(call.message.chat.id, 'Здорово!')
    else:
        bot.send_message(call.message.chat.id, 'Сон - это важно! Не забудьте выспаться сегодня!')
    bot.send_message(call.message.chat.id, 'Опрос завершен, спасибо за участие!')

bot.polling(none_stop=True)
```

Мы изучили далеко не все возможности библиотеки pyTelegrambotAPI. Если хотите использовать другие возможности, изучите ее **[документацию](https://pytba.readthedocs.io/en/latest/types.html#telebot.types.InlineKeyboardMarkup.max_row_keys)**. 